In [20]:
import pandas as pd
import numpy as np
import ast
import re

In [21]:
# 1) Read CSV
df = pd.read_csv(
    "snacks_openfoodfacts.csv",
    dtype={
        "barcode": "str",     # avoid scientific notation
    },
    keep_default_na=True,
    engine="python",
    on_bad_lines="skip"
)



In [22]:
# quick diagnostics
print(df.shape)
print(df.columns.tolist())
print(df.dtypes)
print(df.isna().sum().sort_values(ascending=False).head(30))

(3537, 39)
['url', 'product_name', 'barcode', 'brand', 'quantity', 'serving_size', 'nutriscore_letter', 'nova_group', 'ingredients_text', 'allergens', 'traces', 'energy_kj_100g', 'energy_kcal_100g', 'fat_100g', 'saturated_fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'main_image_url', 'categories', 'contains_palm_oil', 'vegetarian_status', 'vegan_status', 'fruits_vegetables_nuts_percent', 'nutrient_level_fat', 'nutrient_level_saturated_fat', 'nutrient_level_sugars', 'nutrient_level_salt', 'additives', 'packaging', 'stores', 'countries', 'origins', 'manufacturing_places', 'ecoscore_grade', 'ecoscore_score', 'carbon_footprint_100g']
url                                object
product_name                       object
barcode                            object
brand                              object
quantity                           object
serving_size                       object
nutriscore_letter                  object
nova_group           

In [23]:
df.head(3)

,url,product_name,barcode,brand,quantity,serving_size,nutriscore_letter,nova_group,ingredients_text,allergens,...,nutrient_level_salt,additives,packaging,stores,countries,origins,manufacturing_places,ecoscore_grade,ecoscore_score,carbon_footprint_100g
0,https://world.openfoodfacts.org/product/800931...,Mais per Pop Corn – La Casetta di Campagna – 4...,8009320000000.0,La Casetta di Campagna,400 g,NaN,A,NaN,Italian: Mais,NaN,...,[],NaN,NaN,Italy,Argentina,NaN,A,75.0,91.0,NaN
1,https://world.openfoodfacts.org/product/004122...,Raw almonds – H-E-B Organics – 8 oz,41220017751.0,H-E-B Organics,8 oz,NaN,A,NaN,NaN,NaN,...,[],NaN,NaN,United States,NaN,NaN,E,19.0,269.0,NaN
2,https://world.openfoodfacts.org/product/406644...,Linsenwaffeln gesalzen – DmBio – 100g,4066450000000.0,DmBio,100g,NaN,A,3.0,"German: 99,5% Linsenmehl*, 0,5 % Meersalz. aus...",NaN,...,[],NaN,DM,Germany,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# List of the incorrect column names in order
wrong_cols = [
    "fruits_vegetables_nuts_percent",
    "nutrient_level_fat",
    "nutrient_level_saturated_fat",
    "nutrient_level_sugars",
    "nutrient_level_salt",
    "additives",
    "packaging",
    "stores",
    "countries",
    "origins",
    "manufacturing_places",
    "ecoscore_grade",
    "ecoscore_score",
    "carbon_footprint_100g"
]

# Shift column names left by one: remove the first name
correct_cols = wrong_cols[1:]  # everything except the first

# The last column should keep its correct name
# So we append the final intended column name
correct_cols.append("carbon_footprint_100g")

# Now assign the corrected names to the dataframe (only for these columns)
df.rename(columns=dict(zip(wrong_cols, correct_cols)), inplace=True)
df = df.iloc[:, :-1]


In [25]:
df.head()

,url,product_name,barcode,brand,quantity,serving_size,nutriscore_letter,nova_group,ingredients_text,allergens,...,nutrient_level_salt,additives,packaging,stores,countries,origins,manufacturing_places,ecoscore_grade,ecoscore_score,carbon_footprint_100g
0,https://world.openfoodfacts.org/product/800931...,Mais per Pop Corn – La Casetta di Campagna – 4...,8009320000000.0,La Casetta di Campagna,400 g,NaN,A,NaN,Italian: Mais,NaN,...,low,[],NaN,NaN,Italy,Argentina,NaN,A,75.0,91.0
1,https://world.openfoodfacts.org/product/004122...,Raw almonds – H-E-B Organics – 8 oz,41220017751.0,H-E-B Organics,8 oz,NaN,A,NaN,NaN,NaN,...,low,[],NaN,NaN,United States,NaN,NaN,E,19.0,269.0
2,https://world.openfoodfacts.org/product/406644...,Linsenwaffeln gesalzen – DmBio – 100g,4066450000000.0,DmBio,100g,NaN,A,3.0,"German: 99,5% Linsenmehl*, 0,5 % Meersalz. aus...",NaN,...,moderate,[],NaN,DM,Germany,NaN,NaN,NaN,NaN,NaN
3,https://world.openfoodfacts.org/product/007681...,Whole Almonds – Diamond of california – 10 oz,76811342127.0,Diamond of california,10 oz,1/4 cup (30 g) (30 GRM),A,NaN,Almonds,Nuts,...,low,[],NaN,NaN,"United States , World",NaN,NaN,E,19.0,269.0
4,https://world.openfoodfacts.org/product/003068...,Whole Raw Almonds – Tree Of Life Inc.,30684009013.0,Tree Of Life Inc.,NaN,1 ONZ (28 g),A,NaN,Organic whole raw almonds,Nuts,...,low,[],NaN,NaN,United States,NaN,NaN,E,19.0,269.0


In [26]:
df.shape

(3537, 38)

In [27]:
df["barcode"] = df["barcode"].astype(str)
df = df.drop_duplicates(subset="barcode", keep="first")

In [28]:
df.shape

(3367, 38)

In [29]:
# 2) Normalize "unknown"/empty to NaN
df = df.replace(
    ["", "unknown", "Unknown", "NONE", "None", "nan", "NaN"],
    np.nan
)


In [30]:
numeric_cols = [
    "energy_kj_100g", "energy_kcal_100g",
    "fat_100g", "saturated_fat_100g",
    "carbohydrates_100g", "sugars_100g",
    "fiber_100g", "proteins_100g",
    "salt_100g",
    "ecoscore_score",
    "carbon_footprint_100g",
]


In [31]:
#Convert categorical columns to category dtype when appropriate (saves memory)

df["brand"] = df["brand"].astype("category")

In [32]:
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")


In [33]:
pd.set_option("display.max_columns", None)


In [34]:
df.head()

,url,product_name,barcode,brand,quantity,serving_size,nutriscore_letter,nova_group,ingredients_text,allergens,traces,energy_kj_100g,energy_kcal_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,main_image_url,categories,contains_palm_oil,vegetarian_status,vegan_status,nutrient_level_fat,nutrient_level_saturated_fat,nutrient_level_sugars,nutrient_level_salt,additives,packaging,stores,countries,origins,manufacturing_places,ecoscore_grade,ecoscore_score,carbon_footprint_100g
0,https://world.openfoodfacts.org/product/800931...,Mais per Pop Corn – La Casetta di Campagna – 4...,8009320000000.0,La Casetta di Campagna,400 g,NaN,A,NaN,Italian: Mais,NaN,Gluten,1.360,325.0,2.50,0.300,54.00,2.400,13.700,25.00,0.02,https://images.openfoodfacts.org/images/produc...,"Plant-based foods and beverages, Plant-based f...",NaN,yes,yes,low,low,low,low,[],NaN,NaN,Italy,Argentina,NaN,A,75.0,91.0
1,https://world.openfoodfacts.org/product/004122...,Raw almonds – H-E-B Organics – 8 oz,41220017751.0,H-E-B Organics,8 oz,NaN,A,NaN,NaN,NaN,NaN,2.371,566.0,50.00,3.333,20.00,3.333,13.333,20.00,0.00,https://images.openfoodfacts.org/images/produc...,"Plant-based foods and beverages, Plant-based f...",NaN,NaN,NaN,high,moderate,low,low,[],NaN,NaN,United States,NaN,NaN,E,19.0,269.0
2,https://world.openfoodfacts.org/product/406644...,Linsenwaffeln gesalzen – DmBio – 100g,4066450000000.0,DmBio,100g,NaN,A,3.0,"German: 99,5% Linsenmehl*, 0,5 % Meersalz. aus...",NaN,"Sesame seeds, Soybeans",1.523,364.0,1.70,0.400,54.00,1.600,7.600,29.00,0.50,https://images.openfoodfacts.org/images/produc...,"Snacks, Salty snacks",NaN,yes,yes,low,low,low,moderate,[],NaN,DM,Germany,NaN,NaN,NaN,NaN,NaN
3,https://world.openfoodfacts.org/product/007681...,Whole Almonds – Diamond of california – 10 oz,76811342127.0,Diamond of california,10 oz,1/4 cup (30 g) (30 GRM),A,NaN,Almonds,Nuts,NaN,2.370,567.0,50.00,3.330,20.00,3.330,13.300,20.00,0.00,https://images.openfoodfacts.org/images/produc...,"Plant-based foods and beverages, Plant-based f...",NaN,yes,yes,high,moderate,low,low,[],NaN,NaN,"United States , World",NaN,NaN,E,19.0,269.0
4,https://world.openfoodfacts.org/product/003068...,Whole Raw Almonds – Tree Of Life Inc.,30684009013.0,Tree Of Life Inc.,NaN,1 ONZ (28 g),A,NaN,Organic whole raw almonds,Nuts,NaN,2.690,643.0,53.57,3.570,17.86,3.570,10.700,21.43,0.00,https://static.openfoodfacts.org/images/logos/...,"Plant-based foods and beverages, Plant-based f...",NaN,yes,yes,high,moderate,low,low,[],NaN,NaN,United States,NaN,NaN,E,19.0,269.0


In [35]:
missing = (df.isna().sum() / len(df)).sort_values(ascending=False)
print(missing.head(30))

origins                         0.830413
contains_palm_oil               0.822988
manufacturing_places            0.769231
traces                          0.532819
packaging                       0.496287
stores                          0.482031
vegetarian_status               0.449658
serving_size                    0.280071
allergens                       0.272052
vegan_status                    0.246510
quantity                        0.244728
ecoscore_score                  0.209979
ecoscore_grade                  0.209979
carbon_footprint_100g           0.209979
nova_group                      0.181467
fiber_100g                      0.173745
nutriscore_letter               0.049896
brand                           0.047223
ingredients_text                0.047223
nutrient_level_salt             0.027621
nutrient_level_saturated_fat    0.026730
nutrient_level_sugars           0.023463
salt_100g                       0.021978
saturated_fat_100g              0.021384
nutrient_level_f

In [36]:
#negative values and outliers
df.loc[df["sugars_100g"] < 0, "sugars_100g"] = np.nan

# Encoding


In [37]:
# Normalize to uppercase string, "nan" will just become "NAN"
df["contains_palm_oil"] = df["contains_palm_oil"].astype(str).str.strip().str.upper()

# TRUE -> 1, everything else -> 0
df["contains_palm_oil"] = (df["contains_palm_oil"] == "TRUE").astype(int)


In [38]:
level_map = {"low": 0, "moderate": 1, "high": 2}

for col in [
    "nutrient_level_fat",
    "nutrient_level_saturated_fat",
    "nutrient_level_sugars",
    "nutrient_level_salt"
]:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype("string")
            .str.lower()
            .map(level_map)
        )


In [39]:
def count_additives(val):
    if isinstance(val, str) and val.startswith("["):
        try:
            lst = ast.literal_eval(val)
            return len(lst)
        except Exception:
            return np.nan
    return 0

if "additives" in df.columns:
    df["additives_count"] = df["additives"].apply(count_additives)



In [40]:
if "nutriscore_letter" in df.columns:
    df["nutriscore_letter"] = (
        df["nutriscore_letter"]
        .astype("string")
        .str.strip()
        .str.upper()
    )

# Drop rows without label
df = df[~df["nutriscore_letter"].isna()].copy()


In [41]:
# Encode NutriScore Letters (A–E)
if "nutriscore_letter" in df.columns:
    mapping = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5}
    df["nutriscore_letter"] = df["nutriscore_letter"].map(mapping).astype("float")

In [42]:
# Encode Vegetarian Status
if "vegetarian_status" in df.columns:
    veg_map = {"yes": 1, "no": 0}
    df["vegetarian_status"] = df["vegetarian_status"].map(veg_map).astype("float")


In [43]:
# Encode Vegan Status
if "vegan_status" in df.columns:
    vegan_map = {"yes": 1, "no": 0}
    df["vegan_status"] = df["vegan_status"].map(vegan_map).astype("float")


In [44]:
# Encode EcoScore Grade (A–E)
if "ecoscore_grade" in df.columns:
    eco_map = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5}
    df["ecoscore_grade"] = df["ecoscore_grade"].map(eco_map).astype("float")


# Handling missing values

In [45]:
nutr_cols = [
    "energy_kj_100g",
    "energy_kcal_100g",
    "fat_100g",
    "saturated_fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g"
]

# mask rows where all nutrition cols are 0 or NaN
mask_all_empty = (df[nutr_cols].fillna(0) == 0).all(axis=1)

# keep only rows that are NOT all-empty
df = df[~mask_all_empty].copy()


In [46]:
# --- NOVA group ---
if "nova_group" in df.columns:
    # convert to numeric if it's not already
    df["nova_group"] = pd.to_numeric(df["nova_group"], errors="coerce")
    nova_mode = df["nova_group"].mode(dropna=True)
    if not nova_mode.empty:
        df["nova_group"] = df["nova_group"].fillna(nova_mode.iloc[0])

In [47]:
# --- Energy: choose kcal as main, fix from kJ when possible ---
if "energy_kcal_100g" in df.columns and "energy_kj_100g" in df.columns:
    # if kcal missing but kJ present, compute kcal ≈ kJ / 4.184
    mask_missing_kcal = df["energy_kcal_100g"].isna() & df["energy_kj_100g"].notna()
    df.loc[mask_missing_kcal, "energy_kcal_100g"] = df.loc[mask_missing_kcal, "energy_kj_100g"] / 4.184

    # now drop kJ to avoid redundancy
    df = df.drop(columns=["energy_kj_100g"])

In [48]:
numeric_nut_cols = [
    "energy_kcal_100g",
    "fat_100g", "saturated_fat_100g",
    "carbohydrates_100g", "sugars_100g",
    "fiber_100g", "proteins_100g",
    "salt_100g",
]

for col in numeric_nut_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

In [49]:
# --- additives_count ---
if "additives_count" in df.columns:
    df["additives_count"] = df["additives_count"].fillna(0)

In [50]:
df.head()

,url,product_name,barcode,brand,quantity,serving_size,nutriscore_letter,nova_group,ingredients_text,allergens,traces,energy_kcal_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,main_image_url,categories,contains_palm_oil,vegetarian_status,vegan_status,nutrient_level_fat,nutrient_level_saturated_fat,nutrient_level_sugars,nutrient_level_salt,additives,packaging,stores,countries,origins,manufacturing_places,ecoscore_grade,ecoscore_score,carbon_footprint_100g,additives_count
0,https://world.openfoodfacts.org/product/800931...,Mais per Pop Corn – La Casetta di Campagna – 4...,8009320000000.0,La Casetta di Campagna,400 g,NaN,1.0,4.0,Italian: Mais,NaN,Gluten,325.0,2.50,0.300,54.00,2.400,13.700,25.00,0.02,https://images.openfoodfacts.org/images/produc...,"Plant-based foods and beverages, Plant-based f...",0,1.0,1.0,0.0,0.0,0.0,0.0,[],NaN,NaN,Italy,Argentina,NaN,1.0,75.0,91.0,0
1,https://world.openfoodfacts.org/product/004122...,Raw almonds – H-E-B Organics – 8 oz,41220017751.0,H-E-B Organics,8 oz,NaN,1.0,4.0,NaN,NaN,NaN,566.0,50.00,3.333,20.00,3.333,13.333,20.00,0.00,https://images.openfoodfacts.org/images/produc...,"Plant-based foods and beverages, Plant-based f...",0,NaN,NaN,2.0,1.0,0.0,0.0,[],NaN,NaN,United States,NaN,NaN,5.0,19.0,269.0,0
2,https://world.openfoodfacts.org/product/406644...,Linsenwaffeln gesalzen – DmBio – 100g,4066450000000.0,DmBio,100g,NaN,1.0,3.0,"German: 99,5% Linsenmehl*, 0,5 % Meersalz. aus...",NaN,"Sesame seeds, Soybeans",364.0,1.70,0.400,54.00,1.600,7.600,29.00,0.50,https://images.openfoodfacts.org/images/produc...,"Snacks, Salty snacks",0,1.0,1.0,0.0,0.0,0.0,1.0,[],NaN,DM,Germany,NaN,NaN,NaN,NaN,NaN,0
3,https://world.openfoodfacts.org/product/007681...,Whole Almonds – Diamond of california – 10 oz,76811342127.0,Diamond of california,10 oz,1/4 cup (30 g) (30 GRM),1.0,4.0,Almonds,Nuts,NaN,567.0,50.00,3.330,20.00,3.330,13.300,20.00,0.00,https://images.openfoodfacts.org/images/produc...,"Plant-based foods and beverages, Plant-based f...",0,1.0,1.0,2.0,1.0,0.0,0.0,[],NaN,NaN,"United States , World",NaN,NaN,5.0,19.0,269.0,0
4,https://world.openfoodfacts.org/product/003068...,Whole Raw Almonds – Tree Of Life Inc.,30684009013.0,Tree Of Life Inc.,NaN,1 ONZ (28 g),1.0,4.0,Organic whole raw almonds,Nuts,NaN,643.0,53.57,3.570,17.86,3.570,10.700,21.43,0.00,https://static.openfoodfacts.org/images/logos/...,"Plant-based foods and beverages, Plant-based f...",0,1.0,1.0,2.0,1.0,0.0,0.0,[],NaN,NaN,United States,NaN,NaN,5.0,19.0,269.0,0


In [51]:
# --- Handle Missing Values: vegetarian_status ---
if "vegetarian_status" in df.columns:
    # fill missing with 0
    df["vegetarian_status"] = df["vegetarian_status"].fillna(0)


In [52]:
# --- Handle Missing Values: vegan_status ---
if "vegan_status" in df.columns:
    df["vegan_status"] = df["vegan_status"].fillna(0)


In [53]:
# --- Handle Missing Values: ecoscore_grade ---
if "ecoscore_grade" in df.columns:
    mode_val = df["ecoscore_grade"].mode(dropna=True)
    if not mode_val.empty:
        df["ecoscore_grade"] = df["ecoscore_grade"].fillna(mode_val.iloc[0])


In [54]:
# --- Handle Missing Values: ecoscore_score ---
if "ecoscore_score" in df.columns:
    # fill missing with median
    median_val = df["ecoscore_score"].median(skipna=True)
    if pd.notna(median_val):
        df["ecoscore_score"] = df["ecoscore_score"].fillna(median_val)


In [55]:
# --- Handle Missing Values: carbon_footprint_100g ---
if "carbon_footprint_100g" in df.columns:
    # fill missing with median
    median_val = df["carbon_footprint_100g"].median(skipna=True)
    if pd.notna(median_val):
        df["carbon_footprint_100g"] = df["carbon_footprint_100g"].fillna(median_val)


In [56]:
# Columns to handle
nutrient_cols = [
    "nutrient_level_saturated_fat",
    "nutrient_level_sugars",
    "nutrient_level_salt",
    "nutrient_level_fat"
]

for col in nutrient_cols:
    if col in df.columns:

        # Impute missing with mode
        mode_val = df[col].mode(dropna=True)
        if not mode_val.empty:
            df[col] = df[col].fillna(mode_val.iloc[0])

#feature engineering

In [57]:
# Avoid division by zero for carbs
carbs = df['carbohydrates_100g'].replace(0, np.nan)

# 1. Sugar ratio: sugar_g / carbs_g
df['sugar_ratio'] = df['sugars_100g'] / carbs

# Optionally clip to [0, 1] if you want a bounded ratio
# 2. Energy density: kcal per gram (kcal / 1 g)
# energy_kcal_100g is kcal per 100 g → divide by 100
df['energy_density'] = df['energy_kcal_100g'] / 100.0

df["protein_ratio"] = df["proteins_100g"] / (df["carbohydrates_100g"] + 1)


In [58]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [59]:
df["macro_balance"] = (
    df["proteins_100g"] * 0.3
    - df["saturated_fat_100g"] * 0.4
    - df["sugars_100g"] * 0.3
)

In [60]:
df["healthy_score"] = (
    df["fiber_100g"] * 0.25 +
    df["proteins_100g"] * 0.25 -
    df["sugars_100g"] * 0.25 -
    df["saturated_fat_100g"] * 0.25
)

In [61]:
#Log Transform Skewed Features
for col in ["energy_kcal_100g", "fat_100g", "sugars_100g", "salt_100g"]:
    df[f"log_{col}"] = np.log1p(df[col])

In [62]:
# Median imputation
median_val = df["sugar_ratio"].median(skipna=True)
if pd.notna(median_val):
    df["sugar_ratio"] = df["sugar_ratio"].fillna(median_val)


In [63]:
df.shape

(3185, 47)

In [64]:
missing = (df.isna().sum() / len(df)).sort_values(ascending=False)
print(missing.head(30))

origins                  0.826060
manufacturing_places     0.762637
traces                   0.524647
packaging                0.486656
stores                   0.475667
serving_size             0.273783
allergens                0.271272
quantity                 0.246782
brand                    0.045840
ingredients_text         0.043014
countries                0.001256
nutriscore_letter        0.000000
url                      0.000000
product_name             0.000000
barcode                  0.000000
carbohydrates_100g       0.000000
saturated_fat_100g       0.000000
fat_100g                 0.000000
energy_kcal_100g         0.000000
nova_group               0.000000
main_image_url           0.000000
sugars_100g              0.000000
fiber_100g               0.000000
vegetarian_status        0.000000
contains_palm_oil        0.000000
vegan_status             0.000000
nutrient_level_fat       0.000000
nutrient_level_sugars    0.000000
categories               0.000000
proteins_100g 

In [65]:
df.to_csv("preprocessedPhase1FoodFacts.csv", index=False)